# Trading with IBKR

__What can go wrong?__
- API Calls (mostly handled with try/except)
- Connectivity Issues: IBKR very realiable, connect. breaks do not stop the whole session (but the stream...) 

In [ ]:
from ib_insync import * 
import pandas as pd
import numpy as np
import datetime as dt
from IPython.display import display, clear_output
util.startLoop()

__Function for start session__

In [ ]:
def start_session(): 
    global session_start
    
    # take the time of start
    session_start = pd.to_datetime(dt.datetime.utcnow()).tz_localize("utc")
    
    # initialize data streaming (driven data from the market - real time data)
    initialize_stream() 

    # stop the session 
    stop_session() 
    

__Function for initial the stream (get data real time data from market)__ 

In [ ]:
def initialize_stream(): 

    global bars, last_bar # bars data, and the last bar data 
    
    # request the data
    bars = ib.reqHistoricalData(
            contract,             # the instument
            endDateTime='',       
            durationStr='1 D',    # time span
            barSizeSetting=freq,  # bar frequency (trading frequency)
            whatToShow='MIDPOINT',
            useRTH=True,        
            formatDate=2,
            keepUpToDate=True)
    
    # take the date of the last bar
    last_bar = bars[-1].date
    
    # subscribe the function `onBarUpdate()` to the event (get the instument data on real time event).   
    bars.updateEvent += onBarUpdate 


__Callback Function fot catch the real time data__
This function is sibscribed to the event. 

Its mean that this function calling where come new real time data from the server.

In [ ]:
def onBarUpdate(bars, hasNewBar):  
    global df, last_bar
    
    if bars[-1].date > last_bar: 
        last_bar = bars[-1].date
    
        # Data Processing
        df = pd.DataFrame(bars)[["date", "open", "high", "low", "close"]].iloc[:-1] 
        df.set_index("date", inplace = True)
        
        ####################### Trading Strategy ###########################
        df = df[["close"]].copy()
        df["returns"] = np.log(df["close"] / df["close"].shift())
        df["position"] = -np.sign(df.returns.rolling(window).mean())
        ####################################################################
        
        # Trading
        target = df["position"][-1] * units
        execute_trade(target = target)
        
        # Display
        clear_output(wait=True)
        display(df)
    else:
        try:
            trade_reporting()
        except:
            pass

__Function for execute a trad__

In [ ]:
def execute_trade(target):
    global current_pos
    
    # 1. get current Position
    try:
        current_pos = [pos.position for pos in ib.positions() if pos.contract.conId == conID][0]
    except:
        current_pos = 0
         
    # 2. identify required trades
    trades = target - current_pos
        
    # 3. trade execution
    if trades > 0:
        side = "BUY"
        order = MarketOrder(side, abs(trades))
        trade = ib.placeOrder(cfd, order)  
    elif trades < 0:
        side = "SELL"
        order = MarketOrder(side, abs(trades))
        trade = ib.placeOrder(cfd, order)
    else:
        pass


__Function for repert on a trad and give information__

In [ ]:
def trade_reporting():
    global report
    
    fill_df = util.df([fs.execution for fs in ib.fills()])[["execId", "time", "side", "cumQty", "avgPrice"]].set_index("execId")
    profit_df = util.df([fs.commissionReport for fs in ib.fills()])[["execId", "realizedPNL"]].set_index("execId")
    report = pd.concat([fill_df, profit_df], axis = 1).set_index("time").loc[session_start:]
    report = report.groupby("time").agg({"side":"first", "cumQty":"max", "avgPrice":"mean", "realizedPNL":"sum"})
    report["cumPNL"] = report.realizedPNL.cumsum()
        
    clear_output(wait=True)
    display(df, report)


__Function for stop the session__

In [ ]:
def stop_session(): # NEW
    while True:
        ib.sleep(5) # check every 5 seconds
        if dt.datetime.utcnow().time() >= end_time: # if stop conditions has been met
            execute_trade(target = 0) # close open position 
            ib.cancelHistoricalData(bars) # stop stream
            ib.sleep(10)
            try:
                trade_reporting() # final reporting
            except:
                pass
            print("Session Stopped.")
            ib.disconnect()
            break
        else:
            pass

__connection__

In [ ]:
ib = IB()
ib.connect()

__Strategy parameters__

In [ ]:
# strategy parameters
freq = "1 min"
window = 1
units = 1000
end_time = (dt.datetime.utcnow() + dt.timedelta(seconds = 330)).time() # stop condition (5.5 mins from now)
print(end_time)
contract = Forex('EURUSD') 
ib.qualifyContracts(contract)
cfd = CFD("EUR", currency = "USD")
ib.qualifyContracts(cfd)
conID = cfd.conId

__Start the trad__

In [ ]:
start_session()

---

# IBKR API Connectivity Issues (handling)

In [ ]:
from ib_insync import * 
import pandas as pd
import numpy as np
import datetime as dt
from IPython.display import display, clear_output
util.startLoop()

In [ ]:
def start_session():
    global last_update, session_start
    
    last_update = dt.datetime.utcnow() # NEW
    session_start = pd.to_datetime(last_update).tz_localize("utc")
    
    initialize_stream()  
    stop_session()

def initialize_stream(): 
    global bars, last_bar
    
    bars = ib.reqHistoricalData(
            contract,
            endDateTime='',
            durationStr='1 D',
            barSizeSetting=freq,
            whatToShow='MIDPOINT',
            useRTH=True,
            formatDate=2,
            keepUpToDate=True)
    last_bar = bars[-1].date
    bars.updateEvent += onBarUpdate 
    
def onBarUpdate(bars, hasNewBar):  
    global df, last_bar, last_update
    
    last_update = dt.datetime.utcnow() # NEW
    
    if bars[-1].date > last_bar: 
        last_bar = bars[-1].date
    
        # Data Processing
        df = pd.DataFrame(bars)[["date", "open", "high", "low", "close"]].iloc[:-1] 
        df.set_index("date", inplace = True)
        
        ####################### Trading Strategy ###########################
        df = df[["close"]].copy()
        df["returns"] = np.log(df["close"] / df["close"].shift())
        df["position"] = -np.sign(df.returns.rolling(window).mean())
        ####################################################################
        
        # Trading
        target = df["position"][-1] * units
        execute_trade(target = target)
        
        # Display
        clear_output(wait=True)
        display(df)
    else:
        try:
            trade_reporting()
        except:
            pass

def execute_trade(target):
    global current_pos
    
    # 1. get current Position
    try:
        current_pos = [pos.position for pos in ib.positions() if pos.contract.conId == conID][0]
    except:
        current_pos = 0
         
    # 2. identify required trades
    trades = target - current_pos
        
    # 3. trade execution
    if trades > 0:
        side = "BUY"
        order = MarketOrder(side, abs(trades))
        trade = ib.placeOrder(cfd, order)  
    elif trades < 0:
        side = "SELL"
        order = MarketOrder(side, abs(trades))
        trade = ib.placeOrder(cfd, order)
    else:
        pass

def trade_reporting():
    global report
    
    fill_df = util.df([fs.execution for fs in ib.fills()])[["execId", "time", "side", "cumQty", "avgPrice"]].set_index("execId")
    profit_df = util.df([fs.commissionReport for fs in ib.fills()])[["execId", "realizedPNL"]].set_index("execId")
    report = pd.concat([fill_df, profit_df], axis = 1).set_index("time").loc[session_start:]
    report = report.groupby("time").agg({"side":"first", "cumQty":"max", "avgPrice":"mean", "realizedPNL":"sum"})
    report["cumPNL"] = report.realizedPNL.cumsum()
        
    clear_output(wait=True)
    display(df, report)

def stop_session():
    while True:
        ib.sleep(5) 
        if dt.datetime.utcnow().time() >= end_time:
            execute_trade(target = 0) 
            ib.cancelHistoricalData(bars) 
            ib.sleep(10)
            try:
                trade_reporting() 
            except:
                pass
            print("Session Stopped (planned).")
            ib.disconnect()
            break
        elif dt.datetime.utcnow() - last_update > dt.timedelta(seconds=120):
                # if there was no streaming response in the last 120 seconds
                ib.cancelHistoricalData(bars)
                ib.sleep(5)
                try: # try to reestablish stream
                    initialize_stream() # one retry
                except: # stop session
                    ib.sleep(5)
                    try:
                        execute_trade(target = 0) # close open position
                    except:
                        pass
                    ib.sleep(10)
                    try:
                        trade_reporting() # final reporting
                    except:
                        pass
                    print("Session Stopped - No Connection.")
                    ib.disconnect()
                    break
        else:
            pass

In [ ]:
ib = IB()
ib.connect()

In [ ]:
# strategy parameters
freq = "1 min"
window = 1
units = 1000
end_time = (dt.datetime.utcnow() + dt.timedelta(seconds = 330)).time() # stop condition (5.5 mins from now)
contract = Forex('EURUSD') 
ib.qualifyContracts(contract)
cfd = CFD("EUR", currency = "USD")
ib.qualifyContracts(cfd)
conID = cfd.conId

In [ ]:
start_session()